# Calculate Sunset and Sunrise
Welcome to this ugly notebook! What you find here: Calculates the earliest and latest sunrise and sunset for:
- the current year
- if everything was in wintertime
- if everything was in summertime

Earliest sunrise is not during longest day. There may be a fancy formula to calculate this. We go bruth force. So we calculate the sunrise and sunset for each day in year 2025 and find the corresponding entry.

## ToDo
* Add Andorra
* Add Bosnia and Herzegovina
* Isle of man, Jersey, Guernsey
* Nie Nacht prüfen
* Hexagons

In [11]:
import pandas as pd
import geopandas as gpd
from pathlib import Path
from astral import Observer
from astral.sun import sun
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from timezonefinder import TimezoneFinder
import pytz
from tqdm import tqdm
import json

## Defines

In [12]:
path_nuts_eu = Path('../data/nuts3/NUTS_RG_20M_2024_4326.geojson')
path_nuts_uk = Path('../data/nuts3/NUTS_Level_3_January_2018_GCB_in_the_United_Kingdom_2022_-764267078201060548.geojson')
path_boundaries = Path('../data/europeboundaries.geojson')
path_timedata = Path('../export/timedata.json')

## Import Data

In [14]:
# Import Nuts EU
gpd_nuts_eu = gpd.read_file(path_nuts_eu)
gpd_nuts_eu = gpd_nuts_eu[gpd_nuts_eu.LEVL_CODE == 3]

# Remove some countries
gpd_nuts_eu = gpd_nuts_eu[~gpd_nuts_eu.CNTR_CODE.isin(['TR'])]

# Import Nuts UK
gpd_nuts_uk = gpd.read_file(path_nuts_uk)
gpd_nuts_uk.rename(columns={
    'nuts318cd': 'NUTS_ID',
    'nuts318nm': 'NAME_LATN'
    }, inplace=True)
gpd_nuts_uk.to_crs(gpd_nuts_eu.crs, inplace=True)

# Merge
gpd_nuts = pd.concat([gpd_nuts_eu, gpd_nuts_uk], ignore_index=True)
gpd_nuts = gpd_nuts[['NUTS_ID', 'NAME_LATN', 'geometry']]

# Add timezone information to the GeoDataFrame
tf:TimezoneFinder = TimezoneFinder()
gpd_nuts['timezone'] = gpd_nuts.apply(
    lambda row: tf.timezone_at(lat=row.geometry.centroid.y, lng=row.geometry.centroid.x), axis=1
)

# Drop overseas
gdf_boundaries = gpd.read_file(path_boundaries)
gpd_nuts = gpd.overlay(gpd_nuts, gdf_boundaries, how='intersection')

# Save masternuts
gpd_nuts.to_file('../export/masternuts.gpkg')

## Calculate

In [ ]:
# Get all possible dates from 2025
dates = [datetime(2025, 1, 1) + timedelta(days=x) for x in range(365)]

timedata = []

def transform_date_to_winter_summer_time(sun_object, event):

    datetime_local = s[event].astimezone(local_tz)

    # Subtract half a year from local_date
    half_year_before = sun_object[event] - relativedelta(months=6)
    half_year_before = half_year_before.astimezone(local_tz)

    if bool(half_year_before.dst()) == False:
        # Wintertime, no DST
        datetime_winter = half_year_before.time()
        datetime_summer = datetime_local.time()
    else:
        # Summertime, DST
        datetime_winter = datetime_local.time()
        datetime_summer = half_year_before.time()  
       
    return {
            'utc': sun_object[event],
            'local': datetime_local,
            'time_winter': datetime_winter,
            'time_summer': datetime_summer,
        }

def find_earliest_and_latest_in_list(suns_per_day, event, timeperiod):
    return {
            'earliest': min(suns_per_day[event], key=lambda x: x[timeperiod])[timeperiod],
            'earliest_day': min(suns_per_day[event], key=lambda x: x[timeperiod])['local'].date(),
            'latest': max(suns_per_day[event], key=lambda x: x[timeperiod])[timeperiod],
            'latest_day': max(suns_per_day[event], key=lambda x: x[timeperiod])['local'].date()
        }

with tqdm(total=len(gpd_nuts)) as pbar:
    # for i, row in gpd_nuts[gpd_nuts.NAME_LATN == 'Zürich'].iterrows():
    for i, row in gpd_nuts.iterrows():

        pbar.update()

        location = Observer(row.geometry.centroid.y, row.geometry.centroid.x)

        # Earliest sunrise is not the same as the longest day. We need to calculate
        # the earliest day. We could do that the proper way. Or we could just bruth
        # force it. You know what we are gonna do..?
        suns_per_day = {
            'sunrise': [],
            'sunset': []
        }

        local_tz = pytz.timezone(row.timezone)
        for d in dates:
            try:
                s = sun(location, date=d)
                suns_per_day['sunrise'].append(transform_date_to_winter_summer_time(s, 'sunrise'))
                suns_per_day['sunset'].append(transform_date_to_winter_summer_time(s, 'sunset'))
            except ValueError:
                pass

        # Now get the earliest sunrise for winter and summer time
        record = {
            'nuts_id': row['NUTS_ID'],
            'name': row['NAME_LATN'],
            'sunrise': {
                'summer': find_earliest_and_latest_in_list(suns_per_day, 'sunrise', 'time_summer'),
                'winter': find_earliest_and_latest_in_list(suns_per_day, 'sunrise', 'time_winter'),
                'current': {
                    'earliest': min(suns_per_day['sunrise'], key=lambda x: x['local'].time())['local'],
                    'latest': max(suns_per_day['sunrise'], key=lambda x: x['local'].time())['local']
                }
            },
            'sunset': {
                'summer': find_earliest_and_latest_in_list(suns_per_day, 'sunset', 'time_summer'),
                'winter': find_earliest_and_latest_in_list(suns_per_day, 'sunset', 'time_winter'),
                'current': {
                    'earliest': min(suns_per_day['sunset'], key=lambda x: x['local'].time())['local'],
                    'latest': max(suns_per_day['sunset'], key=lambda x: x['local'].time())['local']
                }
            }
        }

        timedata.append(record)

# Store
json.dump(timedata, open(path_timedata, 'w', encoding='UTF-8'), ensure_ascii=False, indent=2, default=str)

print("🍩 Finito")

  0%|          | 0/1425 [00:00<?, ?it/s]

100%|██████████| 1425/1425 [00:25<00:00, 56.93it/s]

🍩 Finito
